In [0]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
np.random.seed(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
sns.set(style='white', context='notebook', palette='deep')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
import math
from __future__ import division
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import model_selection, tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
df = pd.read_csv("/content/gdrive/My Drive/eset_HTA20_probeset_Train_NT.csv")
df = df.transpose()

Work through each 100,000 gene section one at a time. Kind of like a loop. Just uncomment to chunk you want to work on, follow the code below, then come back and do the next chunk.

In [0]:
#X3 = df[df.columns[0:100000]]
#X3 = df[df.columns[100000:200000]]
#X3 = df[df.columns[200000:300000]]
#X3 = df[df.columns[300000:400000]]
#X3 = df[df.columns[400000:500000]]
#X3 = df[df.columns[500000:600000]]
#X3 = df[df.columns[600000:700000]]
#X3 = df[df.columns[700000:800000]]
#X3 = df[df.columns[800000:925030]]
y1 = df[df.columns[925032]]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X3, y1, test_size=0.20, random_state=1)

In [0]:
xgb = xgboost.XGBRegressor(n_estimators=30000, learning_rate=0.15, gamma=0, subsample=1,
                           colsample_bytree=1, max_depth=1)

In [0]:
eval_set = [(X_test, y_test)]
xgb.fit(X_train, y_train, early_stopping_rounds=50, eval_metric="rmse", eval_set=eval_set, verbose=True) 

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[17:57:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:24.2676
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:21.244
[2]	validation_0-rmse:18.7417
[3]	validation_0-rmse:16.5558
[4]	validation_0-rmse:14.7771
[5]	validation_0-rmse:13.3826
[6]	validation_0-rmse:12.1752
[7]	validation_0-rmse:11.2293
[8]	validation_0-rmse:10.4458
[9]	validation_0-rmse:9.74678
[10]	validation_0-rmse:9.22431
[11]	validation_0-rmse:8.8368
[12]	validation_0-rmse:8.47107
[13]	validation_0-rmse:8.17673
[14]	validation_0-rmse:7.96556
[15]	validation_0-rmse:7.74794
[16]	validation_0-rmse:7.66809
[17]	validation_0-rmse:7.58372
[18]	validation_0-rmse:7.5386
[19]	validation_0-rmse:7.46763
[20]	validation_0-rmse:7.39155
[21]	validation_0-rmse:7.37061
[22]	validation_0-rmse:7.23356
[23]	validation_0-rmse:7.20779
[24]	validation_0-rmse:7.14116
[25]	validation_0-rmse:7.09016
[26]	validation

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.15, max_delta_step=0,
             max_depth=1, min_child_weight=1, missing=None, n_estimators=30000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
features = pd.DataFrame(xgb.feature_importances_)

This merges the feature importances with the features

In [0]:
X3 = pd.DataFrame(X3)
X3 = X3.T
features.reset_index(drop=True, inplace=True)
X3.reset_index(drop=True, inplace=True)

frames = [X3, features]
X3 = pd.concat(frames, axis=1)

This drops any features that have a 0 for importance

In [0]:
X3 = X3[(X3 != 0).all(1)]

Find shape to know what column to drop (Dropping the feature importance column)

In [0]:
X3.shape

(179, 368)

We are trying to drop the far right column so from shape above (x, y) use y-1. should be 367 every time.

In [0]:
X3 = X3.drop(X3.columns[367], axis=1)

Flip back to its correct orientation

In [0]:
X3 = X3.T

In [0]:
X3.head() #check it

,241,445,613,1629,2500,2529,2538,3593,4914,6067,6753,8174,8250,8402,10543,10696,10978,11736,12067,12071,12190,13992,14415,16045,16767,17738,19202,19473,19711,20211,20859,21024,22839,24373,24984,25828,26131,27054,27684,27990,...,91035,91180,91263,92730,93089,94163,94940,95169,96081,96205,96889,97101,98004,98420,99072,99172,100615,100616,100632,100846,101525,102328,103365,103908,104364,105031,105131,105198,106271,106397,106759,107050,108305,108309,108770,111807,117320,118513,123319,124855
Tarca_001_P1A01,2.354485,5.677775,5.019519,6.015978,2.022600,5.382738,4.553756,7.944861,2.226139,9.336128,3.269358,2.489024,9.182119,5.461822,5.793349,8.518625,6.739696,2.353764,3.295626,3.849626,2.932390,6.771731,5.033137,3.382666,6.104730,3.054032,6.911797,3.306167,2.419056,7.734427,6.993334,6.670283,8.329305,4.883909,5.315186,6.514435,5.940063,5.957570,5.133054,1.940086,...,3.050201,2.288596,4.224222,3.019242,9.159564,5.601985,6.079564,6.123020,6.334328,8.224543,4.724878,12.546042,2.421585,3.306084,7.521602,4.827859,7.643280,11.877521,3.721569,5.014171,5.813177,5.549163,1.985873,3.678705,4.166612,4.253260,3.563419,7.295166,3.674095,6.378965,6.036418,8.785819,4.482681,2.558984,5.376235,2.293329,2.690024,4.604766,7.433590,2.137929
Tarca_004_P1A04,2.929192,5.255329,6.192424,5.438990,2.585260,6.276500,4.028718,8.057828,2.371291,10.133126,3.402948,1.989408,9.515663,5.253042,5.892292,8.340958,6.278129,1.943941,3.510535,3.467165,2.604930,6.624401,5.008011,2.720975,6.256378,3.384577,7.042094,3.257401,2.575131,7.336377,6.407437,6.491921,7.835327,4.411792,5.427408,6.097974,6.219978,6.075924,4.715674,2.142719,...,3.531077,2.116370,4.272241,2.279641,8.871713,4.900266,6.571698,7.596235,5.690219,8.863236,4.251584,12.638162,2.506464,3.640395,6.585839,5.078295,8.180246,10.335559,3.839184,4.531649,5.572898,5.374108,2.273176,3.534304,3.787985,4.064108,5.155952,6.982794,3.431419,6.185111,5.804158,8.181132,3.903421,3.722905,6.523913,2.016660,2.435822,4.393711,7.953125,1.897539
Tarca_005_P1A05,3.076161,5.362117,5.526927,6.046906,2.446598,5.727366,3.628446,8.179746,2.525952,9.609026,3.212113,2.390696,10.067438,5.160232,5.437918,8.463737,6.432416,2.050187,3.491073,4.114218,2.919216,6.190084,4.378418,3.063216,6.193415,3.502256,7.589840,3.103296,2.287229,8.996055,6.717511,6.419587,8.854956,4.312776,5.428598,6.240501,6.325023,6.080703,5.214818,2.142752,...,2.565858,2.438923,4.352377,4.072907,8.908691,5.017188,6.387343,7.102482,5.290873,8.966486,5.469713,12.655068,2.895704,3.749841,6.729237,5.224319,7.759395,10.325071,3.719830,4.420078,5.479118,5.555716,1.597393,3.649527,3.752035,4.189307,4.847846,6.815268,2.911522,6.583978,5.648801,8.755652,4.776563,3.351324,5.910389,1.984415,2.345699,3.888668,7.860126,2.411225
Tarca_007_P1A07,2.683113,5.489747,5.314329,6.168067,2.212639,6.969993,5.815427,7.884538,2.662123,9.373850,3.966926,2.595548,8.994055,5.783137,5.522076,8.250113,6.336686,2.177656,3.128255,3.687813,2.792401,6.107019,4.641697,3.011967,6.230025,3.607850,7.366155,3.731898,2.591465,7.834151,6.289959,6.567591,8.076642,4.523957,5.227663,6.345817,6.478048,5.753856,5.469109,1.922102,...,2.786146,2.331870,3.600194,3.231818,9.290207,5.769371,6.476821,7.193369,5.832641,9.062117,4.878735,12.696378,4.293860,3.501900,6.520917,4.883814,7.730399,10.419855,3.212946,4.456419,5.847293,5.322456,2.014160,3.629745,3.922476,4.001503,6.738880,6.997063,3.064890,6.460360,5.943007,8.463577,5.568222,3.455297,6.369377,2.150605,2.329852,4.792140,8.212699,2.179327
Tarca_008_P1A08,2.570783,5.296953,5.861601,5.903168,2.392004,6.352775,4.892857,7.614394,2.208451,9.882635,3.283009,2.244929,9.660198,5.084143,4.946701,8.379531,6.235072,2.506147,3.176281,3.626324,2.583422,6.258016,4.694941,2.780831,6.229408,3.522762,7.624183,2.939591,3.634917,6.084426,6.815976,6.512793,7.848821,3.684043,5.439518,6.413924,6.480884,5.932822,5.673679,1.937463,...,3.467918,2.348335,5.035436,1.769182,8.588449,5.498904,6.471847,6.519755,5.520140,9.531953,5.355924,12.573317,3.515404,3.856846,7.197632,5.006447,7.425995,10.2

Save the chunk. we will slap all filtered chunks back together at the end. Give each chunk a unique name

0-100000

In [0]:
o1 = X3.to_csv(r'/content/gdrive/My Drive/01.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

100000-200000

In [0]:
o2 = X3.to_csv(r'/content/gdrive/My Drive/02.csv', index = None, header=True)

200000-300000

In [0]:
o3 = X3.to_csv(r'/content/gdrive/My Drive/03.csv', index = None, header=True)

300000-400000

In [0]:
o4 = X3.to_csv(r'/content/gdrive/My Drive/04.csv', index = None, header=True)

400000-500000

In [0]:
o5 = X3.to_csv(r'/content/gdrive/My Drive/05.csv', index = None, header=True)

500000-600000

In [0]:
o6 = X3.to_csv(r'/content/gdrive/My Drive/06.csv', index = None, header=True)

600000-700000

In [0]:
o7 = X3.to_csv(r'/content/gdrive/My Drive/07.csv', index = None, header=True)

700000-800000

In [0]:
o8 = X3.to_csv(r'/content/gdrive/My Drive/08.csv', index = None, header=True)

800000-925030

In [0]:
o9 = X3.to_csv(r'/content/gdrive/My Drive/09.csv', index = None, header=True)

LOAD THE FILES

In [0]:
X1 = pd.read_csv("/content/gdrive/My Drive/01.csv")
#X2 = pd.read_csv("/content/gdrive/My Drive/02.csv")
X3 = pd.read_csv("/content/gdrive/My Drive/03.csv")
X4 = pd.read_csv("/content/gdrive/My Drive/04.csv")
X5 = pd.read_csv("/content/gdrive/My Drive/05.csv")
X6 = pd.read_csv("/content/gdrive/My Drive/06.csv")
X7 = pd.read_csv("/content/gdrive/My Drive/07.csv")
X8 = pd.read_csv("/content/gdrive/My Drive/08.csv")
X9 = pd.read_csv("/content/gdrive/My Drive/09.csv")

In [0]:
X1 = pd.DataFrame(X1)
#X2 = pd.DataFrame(X2)
X3 = pd.DataFrame(X3)
X4 = pd.DataFrame(X4)
X5 = pd.DataFrame(X5)
X6 = pd.DataFrame(X6)
X7 = pd.DataFrame(X7)
X8 = pd.DataFrame(X8)
X9 = pd.DataFrame(X9)

X1.reset_index(drop=True, inplace=True)
#X2.reset_index(drop=True, inplace=True)
X3.reset_index(drop=True, inplace=True)
X4.reset_index(drop=True, inplace=True)
X5.reset_index(drop=True, inplace=True)
X6.reset_index(drop=True, inplace=True)
X7.reset_index(drop=True, inplace=True)
X8.reset_index(drop=True, inplace=True)
X9.reset_index(drop=True, inplace=True)

In [0]:
frames = [X1, X3, X4, X5, X6, X7, X8, X9]
df = pd.concat(frames, axis = 1)

In [0]:
df = df.T.reset_index(drop=True).T

In [0]:
filteredprobeset15lr = df.to_csv(r'/content/gdrive/My Drive/filteredprobeset15lr.csv', index = None, header=True)

In [0]:
#filteredprobeset15lr.shape
df.shape

(367, 1135)